#### Добавление необходимых путей и импорт библиотек.

In [1]:
import sys
import os
root_path = os.getcwd()
sys.path.append(os.path.join(root_path,'compressai'))
#sys.path

In [2]:
from compressai.zoo import mbt2018_mean
from PIL import Image
from torchvision import transforms
from torchvision.transforms import ToTensor, ToPILImage
import torch
import matplotlib.pyplot as plt
import numpy as np
import glob
import cv2

### В качетсве данных была выбрана поcледовательность пяти кадров из датасета SINTEL (взяты из директории demo-frames из репозитория RAFT), изображения приводим к размеру (H, W) = (512, 1024).

In [4]:
orig_folder = 'images\\demo-frames'
orig_resize_folder = 'images\\orig_resize_frames'
reconstr_folder = 'images\\reconstr_frames'

resize_transform = transforms.Resize((512, 1024))


for filename in os.listdir(orig_folder):
    file_path = os.path.join(orig_folder, filename)
    img = Image.open(file_path).convert('RGB')  

    # Применение трансформации
    resized_img = resize_transform(img)

    
    output_path = os.path.join(orig_resize_folder, filename)
    resized_img.save(output_path)

    print(f'{filename} resized and saved to {orig_resize_folder}')

frame_0021.png resized and saved to images\orig_resize_frames
frame_0022.png resized and saved to images\orig_resize_frames
frame_0023.png resized and saved to images\orig_resize_frames
frame_0024.png resized and saved to images\orig_resize_frames
frame_0025.png resized and saved to images\orig_resize_frames


### Загрузка предобученной модели mbt2018-mean

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net = mbt2018_mean(quality=1, pretrained=True).to(device)
net.eval()


MeanScaleHyperprior(
  (entropy_bottleneck): EntropyBottleneck(
    (likelihood_lower_bound): LowerBound()
  )
  (g_a): Sequential(
    (0): Conv2d(3, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (1): GDN(
      (beta_reparam): NonNegativeParametrizer(
        (lower_bound): LowerBound()
      )
      (gamma_reparam): NonNegativeParametrizer(
        (lower_bound): LowerBound()
      )
    )
    (2): Conv2d(128, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (3): GDN(
      (beta_reparam): NonNegativeParametrizer(
        (lower_bound): LowerBound()
      )
      (gamma_reparam): NonNegativeParametrizer(
        (lower_bound): LowerBound()
      )
    )
    (4): Conv2d(128, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (5): GDN(
      (beta_reparam): NonNegativeParametrizer(
        (lower_bound): LowerBound()
      )
      (gamma_reparam): NonNegativeParametrizer(
        (lower_bound): LowerBound()
      )
    )
    (6): Conv2d(128, 192, kern

### Применение кодека mbt2018-mean к изображениям ( (H, W) = (512, 1024) )

In [6]:

for filename in sorted(os.listdir(orig_resize_folder)):
    
    img_path = os.path.join(orig_resize_folder, filename)
    img = Image.open(img_path).convert('RGB')

    transform = transforms.ToTensor()
    x = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        out_net = net(x)
        
    out_img = out_net['x_hat'].clamp(0, 1).squeeze().cpu()
    out_img_pil = transforms.ToPILImage()(out_img)
    
    output_path = os.path.join(reconstr_folder, f'reconstructed_{filename}')
    out_img_pil.save(output_path)
    
    print(f'Processed and saved {filename}')

Processed and saved frame_0021.png
Processed and saved frame_0022.png
Processed and saved frame_0023.png
Processed and saved frame_0024.png
Processed and saved frame_0025.png


In [7]:

sys.path.append(os.path.join(root_path,'RAFT'))
sys.path.append(os.path.join(root_path,"RAFT\\core"))
#sys.path

In [8]:
from raft import RAFT
from utils import flow_viz
from utils.utils import InputPadder


### Определение путей для сохранения визуализаций оптических потоков

- **opt_flow_between_orig** - Оптические потоки между оригинальными кадрами.
- **pt_flow_between_reconstr** - Оптические потоки между реконструированными кадрами (после применения модели mbt2018-mean).
- **opt_flow_reconstr_orig** - Реконструкции оптических потоков между оригинальными кадрами (после применения модели mbt2018-mean).
- **opt_flow_reconstr_reconstr** - Реконструкции оптических потоков между реконструированными кадрами (после применения модели mbt2018-mean).



In [9]:
opt_flow_between_orig = 'opt_flow\\between_orig'# тут просто оптический поток
opt_flow_between_reconstr = 'opt_flow\\between_reconstr'# тут опт поток между реконстр кадрами
opt_flow_reconstr_orig = 'opt_flow\\reconstr_b-n_orig'# тут реконструкци ориганльных оптических потоков
opt_flow_reconstr_reconstr = 'opt_flow\\reconstr_b-n_reconstr'# тут реконструкция реконструированных оптических потоков


### Вспомогательные функции для использования RAFT.
За основу взят код из файла demo.py из репозитория RAFT

In [10]:
def load_image(imfile):
    img = np.array(Image.open(imfile)).astype(np.uint8)
    img = torch.from_numpy(img).permute(2, 0, 1).float()
    return img[None].to(device)

def demo(args, input_folder, output_folder):
    model = torch.nn.DataParallel(RAFT(args))
    model.load_state_dict(torch.load(args.model))

    model = model.module
    model.to(device)
    model.eval()
    #print("bbbb")
    with torch.no_grad():
        images = glob.glob(os.path.join(input_folder, '*.png'))
        #print("b")
        images = sorted(images)
        for idx, (imfile1, imfile2) in enumerate(zip(images[:-1], images[1:])):
            image1 = load_image(imfile1)
            image2 = load_image(imfile2)
            #print("k")
            padder = InputPadder(image1.shape)
            image1, image2 = padder.pad(image1, image2)

            flow_low, flow_up = model(image1, image2, iters=20, test_mode=True)
           # viz(image1, flow_up)

            opt_flow = flow_up[0].permute(1,2,0).cpu().numpy()
            #print(opt_flow.shape)
            
            # map flow to rgb image
            img_flo = flow_viz.flow_to_image(opt_flow)

            #img_flo = np.concatenate([img, flo], axis=0)
            #import matplotlib.pyplot as plt
            #plt.imshow(img_flo / 255.0)
            #plt.show()
            #import matplotlib.pyplot as plt
            #plt.imshow(img_flo / 255.0)
            #plt.show()

            output_filename = f'flow_{idx}-{idx+1}.png'
            output_path = os.path.join(output_folder, output_filename)
            
            #cv2.imshow('image', img_flo[:, :, [2,1,0]]/255.0)
            #cv2.waitKey()

            cv2.imwrite(output_path, img_flo[:, :, [2,1,0]])
            
            #print('aaa')


### Вычисление оптических потоков между всеми кадрами (сначала между оригинальными, затем между реконструированными) и сохранение визуализаций в указанные директории.

In [11]:
from argparse import Namespace

In [15]:
args1 = Namespace(
    model='models/models/raft-things.pth',
    path=None,
    small=False,
    mixed_precision=False,
    alternate_corr=False
    )
demo(args1, orig_resize_folder, opt_flow_between_orig)
demo(args1, reconstr_folder, opt_flow_between_reconstr)

### Применение кодека mbt2018-mean к визуализациям (к изображениям) оптического потока и сохранение получившихся изображений в указанные директории.

In [16]:
#list_bpp_1 = []
#list_x_hat_1 =[]
for filename in sorted(os.listdir(opt_flow_between_orig)):
    
    img_path = os.path.join(opt_flow_between_orig, filename)
    img = Image.open(img_path).convert('RGB')

    transform = transforms.ToTensor()
    x = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        out_net = net(x)
  
    #list_bpp_1.append(compute_bpp(out_net))

    out_img = out_net['x_hat'].clamp(0, 1).squeeze().cpu()
    out_img_pil = transforms.ToPILImage()(out_img)

    #list_x_hat_1.append(out_net['x_hat'])
    
    output_path = os.path.join(opt_flow_reconstr_orig, f'reconstr_orig_{filename}')
    out_img_pil.save(output_path)
    #print(f'Processed and saved {filename}')

#print(list_x_hat_1)
#formatted_bpp_list = [f"{bpp:.3f}" for bpp in list_bpp_1]
#print(formatted_bpp_list)    
    
#print(likelihoods_list_1)
#list_bpp_2 = []
#list_x_hat_2 = []
for filename in sorted(os.listdir(opt_flow_between_reconstr)):
    
    img_path = os.path.join(opt_flow_between_reconstr, filename)
    img = Image.open(img_path).convert('RGB')

    transform = transforms.ToTensor()
    x = transform(img).unsqueeze(0).to(device)
    
    with torch.no_grad():
        out_net = net(x)
    #list_bpp_2.append(compute_bpp(out_net))
    #list_x_hat_2.append(out_net['x_hat'])

    out_img = out_net['x_hat'].clamp(0, 1).squeeze().cpu()
    out_img_pil = transforms.ToPILImage()(out_img)
    
    output_path = os.path.join(opt_flow_reconstr_reconstr, f'reconstr_reconstr_{filename}')
    out_img_pil.save(output_path)
    
    #print(f'Processed and saved {filename}')
#formatted_bpp_list = [f"{bpp:.3f}" for bpp in list_bpp_2]
#print(formatted_bpp_list)  